# BIA 6304
### Homework 2
### Eric Long

## Code:

In [1]:
#import modules
#code taken from week 3 and 4 examples
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 15000) #important for getting all the text

In [2]:
#read in data source and store in data frame
#data source taken from https://www.kaggle.com/datafiniti/hotel-reviews?select=Datafiniti_Hotel_Reviews_Jun19.csv
#code taken from week 4 examples
pathname = '[redacted]/PERSONAL/Data Science/BIA6304 - Text Mining/Week 4/'
hotel_df = pd.read_csv(pathname + "Datafiniti_Hotel_Reviews_Jun19-hw2_dataset.csv", index_col = False)

#test print lines
#print(hotel_df.shape)
#print(list(hotel_df)) #equivalent of showing the headers of a data frame

In [3]:
#remove everything except review text and rating from data frame
#code taken from week 4 examples as well as https://www.w3resource.com/python-exercises/pandas/python-pandas-data-frame-exercise-5.php
hotel_df = hotel_df[['reviews.text','reviews.rating']]

#test print lines
#print(hotel_df.shape)
#print(list(hotel_df))
#hotel_df.iloc[0:5]

In [4]:
#replace contractions in text
#code taken from week 4 examples (further referenced below)

#code borrowed from http://stackoverflow.com/questions/27845796/replacing-words-matching-regular-expressions-in-python
#import regex package, define replacement patterns, define replacement function
import re

replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]
#the leading "r" = raw string
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s

In [5]:
#run text through the contraction removing function
#also, save the modified data into the dataframe as a new column
#code taken from week 4 examples as well as https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html
replacer = RegexpReplacer()

hotel_df.insert(1,'reviews.text.expanded',hotel_df.loc[:,'reviews.text'])
hotel_df['reviews.text.expanded'] = hotel_df.loc[:,'reviews.text.expanded'].map(lambda x: replacer.replace(x))

#test print lines
#print(hotel_df.shape)
#print(list(hotel_df))
#hotel_df.iloc[0:5]

In [6]:
#make all text lowercase in the "expanded" column
#code taken from https://datatofish.com/lowercase-pandas-dataframe/
hotel_df['reviews.text.expanded'] = hotel_df.loc[:,'reviews.text.expanded'].str.lower()

#test print lines
#print(hotel_df.shape)
#print(list(hotel_df))
#hotel_df.iloc[0:5]

In [7]:
#import nltk stopwords package
#code taken from week 3 examples

from nltk.corpus import stopwords

nltk_stopwords = stopwords.words("english")

#test print lines
#print(type(nltk_stopwords))
#print(len(nltk_stopwords))
#print(nltk_stopwords)

In [8]:
#import vectorizer
#code taken from week 4 examples
from sklearn.feature_extraction.text import CountVectorizer

#look at features without stoplist
cv = CountVectorizer(binary=False, stop_words = None)
cv_no_stop = cv.fit_transform(hotel_df['reviews.text.expanded'])
#print(cv_no_stop.shape)
names = cv.get_feature_names()   #create list of feature names
count = np.sum(cv_no_stop.toarray(), axis = 0) # add up feature counts 
count2 = count.tolist()  # convert numpy array to list
count_df = pd.DataFrame(count2, index = names, columns = ['count']) # create a dataframe from the list
count_df.sort_values(['count'], ascending = False)[0:19]  #arrange by count instead of alphabetical (top 20)

#print(count_df.shape)

,count
the,67993
and,42699
to,36259
was,23482
we,22108
you,18577
in,17617
is,17399
for,17169
of,14938


In [9]:
#look at features using stoplist
cv2 = CountVectorizer(binary=False, stop_words = nltk_stopwords)
cv_stop = cv2.fit_transform(hotel_df['reviews.text.expanded'])
#print(cv_stop.shape)
new_names = cv2.get_feature_names()   #create list of feature names
new_count = np.sum(cv_stop.toarray(), axis = 0) # add up feature counts 
new_count2 = new_count.tolist()  # convert numpy array to list
new_count_df = pd.DataFrame(new_count2, index = new_names, columns = ['new_count']) # create a dataframe from the list
new_count_df.sort_values(['new_count'], ascending = False)[0:19]  #arrange by count instead of alphabetical (top 20)

,new_count
hotel,12370
room,10282
stay,7890
staff,6250
great,5909
us,5638
clean,4222
would,4175
breakfast,4174
thank,3961


In [10]:
#add additional words to stoplist
#code taken from week 4 examples

new_stopwords = nltk_stopwords + ["hotel","room","stay","staff","us","would","rooms","location","place","breakfast","area","night"]

#look at updated feature list
cv3 = CountVectorizer(binary=False, stop_words = new_stopwords)
cv_stop_add = cv3.fit_transform(hotel_df['reviews.text.expanded'])
#print(cv_stop_add.shape)
newest_names = cv3.get_feature_names()   #create list of feature names
newest_count = np.sum(cv_stop_add.toarray(), axis = 0) # add up feature counts 
newest_count2 = newest_count.tolist()  # convert numpy array to list
newest_count_df = pd.DataFrame(newest_count2, index = newest_names, columns = ['newest_count']) # create a dataframe from the list
newest_count_df.sort_values(['newest_count'], ascending = False)[0:19]  #arrange by count instead of alphabetical (top 20)

,newest_count
great,5909
clean,4222
thank,3961
nice,3919
time,3837
good,3637
one,3008
stayed,2935
friendly,2817
service,2768


In [11]:
#remove stopwords from dataframe
#code taken from week 3 examples and https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe

hotel_df['reviews.text.expanded'] = hotel_df['reviews.text.expanded'].apply(lambda x: [y for y in x.split() if y not in new_stopwords])
hotel_df['reviews.text.expanded'] = [" ".join(y) for y in hotel_df['reviews.text.expanded']]
#hotel_df['reviews.text.expanded'].head(5)

In [12]:
#test print lines
#print(hotel_df.shape)
#print(list(hotel_df))
hotel_df.iloc[0:5]

,reviews.text,reviews.text.expanded,reviews.rating
0,"This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.","nice quiet. know, train track near by. train passed stay. best western changed classification. plus category before.",3
1,"We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More","stayed king suite separation bedroom living space. sofa bed good back discomfort day left three stay. clean, king bed comfortable. located within walking distance places want to...",4
2,"Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.","parking horrible, somebody ran rental car staying there. get try breakfast, business restaurant opened late business world enjoy, asked coffee room, items vending machine stale.",3
3,"Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental breakfast which is a plus and compensates. Front desk service and personnel where excellent. It is Carmel, no A/C in rooms but they have a fan for air circulation.","cheap excellent location. price somewhat standard hacing reservations. nice clean. offer good continental plus compensates. front desk service personnel excellent. carmel, a/c fan air circulation.",5
4,"If you get the room that they advertised on the website and for what you paid, you may be lucky.If you stay many days , they will give you the not so good rooms.Nobody wants to stay in these rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, or difficult access. If you stay one-two days you will get probably... More","get advertised website paid, may lucky.if many days , give good rooms.nobody wants rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, difficult access. one-two days get probably...",2


In [13]:
#use AFINN for sentiment analysis
#code taken from week 4 examples
afinn = {}
for line in open(pathname+"AFINN-111.txt"):
    tt = line.split('\t')
    afinn.update({tt[0]:int(tt[1])})


#test print lines
#print(type(afinn), len(afinn))

#for key, value in sorted(afinn.items())[0:10]:
#    print(key + " => " + str(value))
#print("~~~~~~~~~~~~")
#for key, value in sorted(afinn.items())[2467:]:
#    print(key + " => " + str(value))

In [14]:
#define AFINN function
#code taken from week 4 examples

def afinn_sent(inputstring):
    
    sentcount =0
    for word in inputstring.split():  
        if word.rstrip('?:!.,;') in afinn:
            sentcount = sentcount + afinn[word.rstrip('?:!.,;')]
            
    
    if (sentcount < 0):
        sentiment = 'Negative'
    elif (sentcount > 0):
        sentiment = 'Positive'
    else:
        sentiment = 'Neutral'
    
    return sentiment

In [15]:
#pass dataframe reviews into sentiment analysis function
#code taken from week 4 examples

hotel_df['afinn'] = hotel_df['reviews.text.expanded'].apply(lambda x: afinn_sent(x))

#test print lines
#print(type(hotel_df.afinn))
hotel_df.iloc[0:5]

,reviews.text,reviews.text.expanded,reviews.rating,afinn
0,"This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.","nice quiet. know, train track near by. train passed stay. best western changed classification. plus category before.",3,Positive
1,"We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More","stayed king suite separation bedroom living space. sofa bed good back discomfort day left three stay. clean, king bed comfortable. located within walking distance places want to...",4,Positive
2,"Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.","parking horrible, somebody ran rental car staying there. get try breakfast, business restaurant opened late business world enjoy, asked coffee room, items vending machine stale.",3,Negative
3,"Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental breakfast which is a plus and compensates. Front desk service and personnel where excellent. It is Carmel, no A/C in rooms but they have a fan for air circulation.","cheap excellent location. price somewhat standard hacing reservations. nice clean. offer good continental plus compensates. front desk service personnel excellent. carmel, a/c fan air circulation.",5,Positive
4,"If you get the room that they advertised on the website and for what you paid, you may be lucky.If you stay many days , they will give you the not so good rooms.Nobody wants to stay in these rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, or difficult access. If you stay one-two days you will get probably... More","get advertised website paid, may lucky.if many days , give good rooms.nobody wants rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, difficult access. one-two days get probably...",2,Positive


In [16]:
#use HL for sentiment analysis
#code taken from week 4 examples

HLpos = [line.strip() for line in  open(pathname+'HLpos.txt','r')]
HLneg = [line.strip() for line in  open(pathname +'HLneg.txt','r',encoding = 'latin-1')]
#print("HL pos  size: " + str(len(HLpos)))
#print(HLpos[0:10])
#print("HL neg  size: " + str(len(HLneg)))
#print(HLneg[0:10])

In [17]:
#define HL function
#code taken from week 4 examples

def hl_sent(inputstring):

    poscount = 0
    negcount = 0
    
    for word in inputstring.split(): 
        if HLpos.count(word.rstrip('?:!.,;')):
            poscount +=1
        elif HLneg.count(word.rstrip('?:!.,;')):
            negcount +=1
     
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))    
    else:
        t = 0
    
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

In [18]:
#apply HL sentiment analysis to dataframe
hotel_df['hl'] = hotel_df['reviews.text.expanded'].apply(lambda x: hl_sent(x))
hotel_df.iloc[0:5][['reviews.text.expanded','reviews.rating','afinn','hl']]

,reviews.text.expanded,reviews.rating,afinn,hl
0,"nice quiet. know, train track near by. train passed stay. best western changed classification. plus category before.",3,Positive,Positive
1,"stayed king suite separation bedroom living space. sofa bed good back discomfort day left three stay. clean, king bed comfortable. located within walking distance places want to...",4,Positive,Positive
2,"parking horrible, somebody ran rental car staying there. get try breakfast, business restaurant opened late business world enjoy, asked coffee room, items vending machine stale.",3,Negative,Negative
3,"cheap excellent location. price somewhat standard hacing reservations. nice clean. offer good continental plus compensates. front desk service personnel excellent. carmel, a/c fan air circulation.",5,Positive,Positive
4,"get advertised website paid, may lucky.if many days , give good rooms.nobody wants rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, difficult access. one-two days get probably...",2,Positive,Negative


In [19]:
#convert number rating to afinn/hl format of neg, neutral, or pos
#code taken from https://stackoverflow.com/questions/44991438/lambda-including-if-elif-else
#and https://datatofish.com/if-condition-in-pandas-dataframe/

#rating of 3 == neutral, < 3 == negative, > 3 == positive
hotel_df['actual.rating'] = hotel_df['reviews.rating'].apply(lambda x: 'Positive' if x > 3 else ('Negative' if x < 3 else 'Neutral'))

#final table
hotel_df.iloc[0:5]

,reviews.text,reviews.text.expanded,reviews.rating,afinn,hl,actual.rating
0,"This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.","nice quiet. know, train track near by. train passed stay. best western changed classification. plus category before.",3,Positive,Positive,Neutral
1,"We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More","stayed king suite separation bedroom living space. sofa bed good back discomfort day left three stay. clean, king bed comfortable. located within walking distance places want to...",4,Positive,Positive,Positive
2,"Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.","parking horrible, somebody ran rental car staying there. get try breakfast, business restaurant opened late business world enjoy, asked coffee room, items vending machine stale.",3,Negative,Negative,Neutral
3,"Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental breakfast which is a plus and compensates. Front desk service and personnel where excellent. It is Carmel, no A/C in rooms but they have a fan for air circulation.","cheap excellent location. price somewhat standard hacing reservations. nice clean. offer good continental plus compensates. front desk service personnel excellent. carmel, a/c fan air circulation.",5,Positive,Positive,Positive
4,"If you get the room that they advertised on the website and for what you paid, you may be lucky.If you stay many days , they will give you the not so good rooms.Nobody wants to stay in these rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, or difficult access. If you stay one-two days you will get probably... More","get advertised website paid, may lucky.if many days , give good rooms.nobody wants rooms: low light/dark rooms, near pool, noisy, smelly bathrooms, difficult access. one-two days get probably...",2,Positive,Negative,Negative


In [20]:
#show sentiment analysis stats
#code taken from https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html
#and https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

stats_df = pd.DataFrame()
stats_df['actual'] = hotel_df['actual.rating'].value_counts()
stats_df['afinn'] = hotel_df['afinn'].value_counts()
stats_df['hl'] = hotel_df['hl'].value_counts()

#output:
stats_df.iloc[0:10]

,actual,afinn,hl
Positive,7689,9236,8965
Neutral,1190,266,376
Negative,1121,498,659


In [21]:
#export new dataframe for review
#code taken from https://stackabuse.com/reading-and-writing-csv-files-in-python-with-pandas/

hotel_df.to_csv(pathname + "hotel_df.csv")

## Questions

#### Q1.  Describe what preprocessing (stop words, stemming, custom replacement, etc.) you performed to make the data more useful.  Make sure your answer is no longer than two concise paragraphs and tailored towards a technical audience. At a minimum your response should answer these questions:

##### What preprocessing steps did you perform and why?
##### Did your preprocessing yield the expected results? Support your stance with examples?
##### What ‘mishaps’ did you observe and how might you adapt your decisions/methods to account for them?

&emsp;The preprocessing steps performed include converting contractions (for example, "I'd" to "I would") and removing stop words. The stop word list used was NLTK. Contractions were removed to make the text standardized for sentiment analysis. The stop words were removed to allow less processing when performing sentiment analysis. The words removed generally do not have an impact on sentiment. I am not sure the preprocessing had a huge impact on determining sentiment since the changes just removed words that are not evaluated in the first place. There are some areas where it could cause an issue based on certain phrases in the sentiment dictionary. For example, "can't stand" has a -3 sentiment in AFINN so it potentially could have been missed after my preprocessing. Outside of code related issues I did not run into many mishaps. I intentionally kept preprocessing to a minimum, instead focusing on two sentiment dictionaries to see how different the results may be.

#### Q2. Write a short description of how the sentiment analysis was done and what the outcome is. Make sure your answer is no longer than three paragraphs and tailored to a non-technical audience.  At a minimum answer these questions:

##### What measure did you use to determine the sentiment label? Why?
##### Which version of your text yielded the most useful labels? Why?  Support your stance with specific examples.
##### What changes (to your text, dictionary, or sentiment measurement) could you take to make your labels more useful?

&nbsp; I used two different measures for sentiment analysis. The first was the AFINN dictionary, followed by the HL dictionary. I chose to use two so I could compare the results with the actual review ratings. AFINN is considerably smaller than HL, so there could be significant differences. As expected, the HL analysis with the larger dictionary provided results more in-line with the actual review ratings. The HL method resulted in 8965 positive reviews compared to 7689 original positive ratings. However, both AFINN and HL runs resulted in a lot of misses for neutral and negative comments. Therefore, in terms of usefulness I am not sure I'd consider any of the results particularly great.

&nbsp; I think there are a couple of different things I could have done to produce more accurate results. First, adding more specific "hotel" words to the sentiment dictionaries would help. More specifically, I think some multiple word phrases would have been helpful. For example, the word "bed" might not be significant but "bed comfortable" would be a positive comment. Another one is "location"; this doesn't sound like much but some of the surronding words provide some insight. Also, in retrospect I should have performed sentiment analysis on the original text (before preprocessing) to see how close the calculated results matched with the original ratings. This would provide a better baseline for comparing the preprocessed text.